In [4]:
import os
mingw_path = 'C:\\mingw-w64\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
import xgboost as xgb

In [ ]:
feature_list = ['x','y','hour','weekday', 'day','month','year', 'accuracy']

In [5]:
def prepare_data(df):    
    #Feature engineering
    df.x = df.x.values
    df.y = df.y.values
    initial_date = np.datetime64('2014-01-02T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = (d_times.hour+ d_times.minute/60)
    #for i in range(0,24):
    #    df['h' + str(i)] = (((d_times.hour+ d_times.minute/60) + i) % 24)
    
    #df['w0'] = ((d_times.weekday + 0) % 7)
    #df['w1'] = ((d_times.weekday + 1) % 7)
    #df['w2'] = ((d_times.weekday + 2) % 7)
    #df['w3'] = ((d_times.weekday + 3) % 7)
    #df['w4'] = ((d_times.weekday + 4) % 7)
    #df['w5'] = ((d_times.weekday + 5) % 7)
    #df['w6'] = ((d_times.weekday + 6) % 7)
    df['weekday'] = d_times.weekday
    
    df['day'] = (d_times.dayofyear).astype(int)
    df['month'] = d_times.month
    df['year'] = (d_times.year - 2013)
    #df.accuracy = df.accuracy.values * fw[7]
    df['accuracy'] = np.log10(df.accuracy)
    df['log_month'] = np.log10(3+df.time/(60 * 24 * 30))
    #df = df.drop(['time'], axis=1)
    
    return df

In [6]:
import numpy as np
import pandas as pd
import xgboost
from sklearn import preprocessing

NBINS=2 # Randomly pick this number of bins and fit
# See line 92 for iterating over the whole dataset

def horizontally_bin_data(data, NX, NY):
    """Add columns to data indicating X and Y bins.

    Divides the grid into `NX` bins in X and `NY` bins in Y, and adds columns 
    to `data` containing the bin number in X and Y. 
    """

    NX = int(NX)
    NY = int(NY)

    assert((NX >= 5) and (NX <= 1000))
    assert((NY >= 5) and (NY <= 1000))

    x_bounds = (0., 10.)
    y_bounds = (0., 10.)

    delta_X = (x_bounds[1] - x_bounds[0]) / float(NX)
    delta_Y = (y_bounds[1] - y_bounds[0]) / float(NY)

    # very fast binning algorithm, just divide by delta and round down
    xbins = np.floor((data.x.values - x_bounds[0])
                     / delta_X).astype(np.int32)
    ybins = np.floor((data.y.values - y_bounds[0])
                     / delta_Y).astype(np.int32)

    # some points fall on the upper/right edge of the domain
    # tweak their index to bring them back in the box
    xbins[xbins == NX] = NX-1
    ybins[ybins == NY] = NY-1

    xlabel = 'x_bin_{0:03d}'.format(NX)
    ylabel = 'y_bin_{0:03d}'.format(NY)

    data[xlabel] = xbins
    data[ylabel] = ybins
    return


def mapkprecision(truthvalues, predictions):
    '''
    This is a faster implementation of MAP@k valid for numpy arrays.
    It is only valid when there is one single truth value. 

    m ~ number of observations
    k ~ MAP at k -- in this case k should equal 3

    truthvalues.shape = (m,) 
    predictions.shape = (m, k)
    '''
    z = (predictions == truthvalues[:, None]).astype(np.float32)
    weights = 1./(np.arange(predictions.shape[1], dtype=np.float32) + 1.)
    z = z * weights[None, :]
    return np.mean(np.sum(z, axis=1))


def CV_fit(train, test, kfold=5, keep_fraction=0.5):
    '''Performs a fit using XGBoost. Applies kfold cross validation to 
    estimate error. 

    Parameters
    ----------
    train 
        The training dataset as a pandas DataFrame
    test
        The testing dataset as a pandas DataFrame
    kfold
        The number of folds to use for K Fold Validation
    keep_fraction
        A float between 0 and 1. The fraction of events in each bin
        to keep while minimizing the number of place_ids in the training
        set. Low values throw away a lot of infrequent place_ids, and
        values near 1 retain almost all place_ids. 
    '''

    # choose 10 random bins for a quicker estimate of algorithm error.
    rs = np.random.RandomState(42)
    #bin_numbers = zip(rs.randint(0, 50, size=NBINS), rs.randint(0, 50, size=NBINS))
    bin_numbers[]
    for(int i in range(0, ))

    predictions = []
    map3s = []

    #Choose this line for the whole dataset.
    #for i_bin_x, i_bin_y in itertools.product(xrange(50), xrange(50)):
    for i_bin_x, i_bin_y in bin_numbers:
        print("Bin {},{}".format(i_bin_x, i_bin_y))

        # choose the correct bin, sort values in time to better simulate
        # the real train/test split for k-fold validation
        train_in_bin = train[(train.x_bin_050 == i_bin_x)
                             & (train.y_bin_050 == i_bin_y)].sort_values('time')
        test_in_bin = test[(test.x_bin_050 == i_bin_x)
                           & (test.y_bin_050 == i_bin_y)].sort_values('time')

        N_total_in_bin = train_in_bin.shape[0]
        keep_N = int(float(N_total_in_bin)*keep_fraction)
        vc = train_in_bin.place_id.value_counts()

        # eliminate all ids which are low enough frequency
        vc = vc[np.cumsum(vc.values) < keep_N]
        df1 = pd.DataFrame({'place_id': vc.index, 'freq': vc.values})

        # this represents the training set after all low frequency place_ids
        # are removed
        train_in_bin_2 = pd.merge(train_in_bin, df1, on='place_id',
                                  how='inner')

        # XG Boost requires labels from 0 to n_labels, not place_ids
        le = preprocessing.LabelEncoder()
        le.fit(train_in_bin_2.place_id.values)
        y_train = le.transform(train_in_bin_2.place_id.values)

        # select columns (features) and make a numpy array
        x_train = train_in_bin_2[feature_list].as_matrix()
        x_test = test_in_bin[feature_list].as_matrix()

        # Construct DMatrices
        dm_train = xgboost.DMatrix(x_train, label=y_train)
        dm_test = xgboost.DMatrix(x_test)

        # use the XGBoost built in cross validation function,
        # stopping early to prevent overfitting
        res = xgboost.cv(
            {'eta': 0.1, 'objective': 'multi:softprob',
             'num_class': len(le.classes_),
             'alpha': 0.1, 'lambda': 0.1, 'booster': 'gbtree'},
            dm_train, num_boost_round=200, nfold=kfold, seed=42,
            early_stopping_rounds=10, verbose_eval=10
            # For some reason, verbose_eval seems to be broken on my install
        )

        print(res)

        # this will be the number of epochs that (approximately) prevents
        # overfitting
        N_epochs = res.shape[0]

        # For some reason, verbose_eval seems to be broken on my install
        booster = xgboost.train(
            {'eta': 0.1, 'objective': 'multi:softprob',
             'num_class': len(le.classes_),
             'alpha': 0.1, 'lambda': 0.1, 'booster': 'gbtree'},
            dm_train, num_boost_round=N_epochs, verbose_eval=10)
        predict_y_train = booster.predict(dm_train)
        predict_y_test = booster.predict(dm_test)

        # There is a DELIBERATE error here where the CV MAP@3 is misleadingly
        # good as compared to the true MAP@3 on the test set. I leave it as
        # an exercise how to fix this.

        # A top k algorithm would be theoretically faster, but benchmarks
        # indicate that with Order(50) elements, an argsort is faster than
        # heapq.

        predicted_train_idx = np.argsort(
            predict_y_train, axis=1)[:, -3:][:, ::-1]
        predicted_test_idx = np.argsort(
            predict_y_test, axis=1)[:, -3:][:, ::-1]

        c = np.array(le.classes_)
        predicted_train_place_id = np.take(c, predicted_train_idx)
        predicted_test_place_id = np.take(c, predicted_test_idx)

        map3 = mapkprecision(c.take(y_train), predicted_train_place_id)
        map3s.append(map3)

        print("Train Cross Validated MAP@3: {0:.4f}".format(map3))
        result = pd.DataFrame({'row_id': test_in_bin.index,
                               'pred_1': predicted_test_place_id[:, 0],
                               'pred_2': predicted_test_place_id[:, 1],
                               'pred_3': predicted_test_place_id[:, 2]})
        predictions.append(result)

    return pd.concat(predictions), map3s


def main():

    print('Reading csv files from disk.', flush=True)
    train = pd.read_csv('../../train.csv')
    test = pd.read_csv('../../test.csv')

    print("Calculating time features", flush=True)

    #train['hour'] = (train['time']//60) % 24+1  # 1 to 24
    #test['hour'] = (test['time']//60) % 24+1  # 1 to 24
    
    train = prepare_data(train)
    test = prepare_data(test)
    
    # You'll probably want to generate more features here.

    print("Binning data", flush=True)
    horizontally_bin_data(train, 50, 50)
    horizontally_bin_data(test, 50, 50)

    print("Starting CV", flush=True)
    predictions, map3s = CV_fit(train, test, kfold=5)
    print("MAP@3 CV in bins {}".format(map3s), flush=True)

    print("Writing xgb_submission.csv", flush=True)
    with open('xgb_submission.csv', 'w') as fh:
        fh.write('row_id,place_id\n')
        for r in predictions.itertuples():
            fh.write("{0},{1} {2} {3}\n".format(*r))

if __name__ == '__main__':
    main()

Reading csv files from disk.
Calculating time features
Binning data
Starting CV
Bin 38,14
[0]	train-merror:0.126267+0.00357539	test-merror:0.178857+0.00648689
[10]	train-merror:0.101079+0.00377701	test-merror:0.171737+0.00828306
[20]	train-merror:0.0892126+0.00178405	test-merror:0.169148+0.0105121
[30]	train-merror:0.0782092+0.00124187	test-merror:0.164617+0.0114693
[40]	train-merror:0.0676914+0.00146723	test-merror:0.164401+0.0105388
[50]	train-merror:0.0576592+0.000711339	test-merror:0.164186+0.00634188
[60]	train-merror:0.0497306+0.00203702	test-merror:0.16397+0.00491968
[70]	train-merror:0.0401832+0.0019147	test-merror:0.163538+0.00494822
    test-merror-mean  test-merror-std  train-merror-mean  train-merror-std
0           0.178857         0.006487           0.126267          0.003575
1           0.177562         0.004948           0.119417          0.002846
2           0.176268         0.004948           0.115318          0.001659
3           0.175405         0.005311           0

In [8]:
print('Reading csv files from disk.', flush=True)
train = pd.read_csv('../../train.csv')
test = pd.read_csv('../../test.csv')

print("Calculating time features", flush=True)

#train['hour'] = (train['time']//60) % 24+1  # 1 to 24
#test['hour'] = (test['time']//60) % 24+1  # 1 to 24

train = prepare_data(train)
test = prepare_data(test)

# You'll probably want to generate more features here.

print("Binning data", flush=True)
horizontally_bin_data(train, 50, 50)
horizontally_bin_data(test, 50, 50)

Reading csv files from disk.
Calculating time features
Binning data


In [9]:
train.head()

,row_id,x,y,accuracy,time,place_id,hour,weekday,day,month,year,log_month,x_bin_050,y_bin_050
0,0,0.7941,9.0809,1.732394,470702,8523065625,6.050000,1,329,11,1,1.142886,3,45
1,1,5.9567,4.7968,1.113943,186555,1757726713,22.266667,6,131,5,1,0.864416,29,23
2,2,8.3078,7.0407,1.869232,322648,1137537235,10.483333,3,226,8,1,1.019893,41,35
3,3,7.3665,2.5165,1.812913,704587,6567393236,16.133333,2,126,5,2,1.285780,36,12
4,4,4.0961,1.1307,1.491362,472130,7440663949,5.850000,2,330,11,1,1.143918,20,5
